In [27]:
import pandas as pd
from IPython.core.display_functions import display
from pandas import DataFrame, json_normalize
from utils import num_of_round, is_allowed_round, response_from_json

FILE_NAME = 'data/league39_season2018.json'
response = response_from_json(file_name=FILE_NAME)

In [28]:
all_response_data = json_normalize(data=response)
all_response_data = all_response_data.astype(
    {'goals.away': 'Int64', 'goals.home': 'Int64'}, errors='ignore')
all_response_data

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,...,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
0,65,"Andre Marriner, England",UTC,2018-08-10T19:00:00+00:00,1533927600,1533927600,1533931200,556.0,Old Trafford,Manchester,...,2,1,1,0,2,1,None,None,None,None
1,66,"Martin Atkinson, England",UTC,2018-08-11T11:30:00+00:00,1533987000,1533987000,1533990600,562.0,St James' Park,Newcastle upon Tyne,...,1,2,1,2,1,2,None,None,None,None
2,67,"Kevin Friend, England",UTC,2018-08-11T14:00:00+00:00,1533996000,1533996000,1533999600,NaN,Vitality Stadium,Bournemouth,...,2,0,1,0,2,0,None,None,None,None
3,68,"Mike Dean, England",UTC,2018-08-11T14:00:00+00:00,1533996000,1533996000,1533999600,535.0,Craven Cottage,London,...,0,2,0,1,0,2,None,None,None,None
4,69,"Chris Kavanagh, England",UTC,2018-08-11T14:00:00+00:00,1533996000,1533996000,1533999600,NaN,John Smith's Stadium,Huddersfield,...,0,3,0,2,0,3,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,440,"Martin Atkinson, England",UTC,2019-05-12T14:00:00+00:00,1557669600,1557669600,1557673200,550.0,Anfield,Liverpool,...,2,0,1,0,2,0,None,None,None,None
376,441,"Jonathan Moss, England",UTC,2019-05-12T14:00:00+00:00,1557669600,1557669600,1557673200,556.0,Old Trafford,Manchester,...,0,2,0,1,0,2,None,None,None,None
377,442,"Lee Probert, England",UTC,2019-05-12T14:00:00+00:00,1557669600,1557669600,1557673200,NaN,St. Mary's Stadium,Southampton,...,1,1,1,0,1,1,None,None,None,None
378,443,"Andre Marriner, England",UTC,2019-05-12T14:00:00+00:00,1557669600,1557669600,1557673200,593.0,Tottenham Hotspur Stadium,London,...,2,2,1,0,2,2,None,None,None,None


In [29]:
COLUMN_NAMES_DICT = {'teams.home.name': 'host',
                     'teams.away.name': 'guest',
                     'goals.home': 'goals_host',
                     'goals.away': 'goals_guest',
                     'league.round': 'round'}

all_games = all_response_data[COLUMN_NAMES_DICT.keys()]
all_games = all_games.rename(columns=COLUMN_NAMES_DICT)
all_games['round'] = all_games['round'].apply(lambda s: num_of_round(s))
all_games

,host,guest,goals_host,goals_guest,round
0,Manchester United,Leicester,2,1,1
1,Newcastle,Tottenham,1,2,1
2,Bournemouth,Cardiff,2,0,1
3,Fulham,Crystal Palace,0,2,1
4,Huddersfield,Chelsea,0,3,1
...,...,...,...,...,...
375,Liverpool,Wolves,2,0,38
376,Manchester United,Cardiff,0,2,38
377,Southampton,Huddersfield,1,1,38
378,Tottenham,Everton,2,2,38


In [30]:
games_allowed = all_games[all_games["round"].apply(lambda n: is_allowed_round(n))]
games_allowed

,host,guest,goals_host,goals_guest,round
60,West Ham,Manchester United,3,1,7
61,Arsenal,Watford,2,0,7
62,Everton,Fulham,3,0,7
63,Huddersfield,Tottenham,0,2,7
64,Manchester City,Brighton,2,0,7
...,...,...,...,...,...
315,Fulham,Manchester City,0,2,32
316,Leicester,Bournemouth,2,0,32
317,Liverpool,Tottenham,2,1,32
318,Manchester United,Watford,2,1,32


In [31]:
games_allowed[ games_allowed['round'] == 8 ]

,host,guest,goals_host,goals_guest,round
70,Brighton,West Ham,1,0,8
71,Burnley,Huddersfield,1,1,8
72,Crystal Palace,Wolves,0,1,8
73,Fulham,Arsenal,1,5,8
74,Leicester,Everton,1,2,8
75,Liverpool,Manchester City,0,0,8
76,Manchester United,Newcastle,3,2,8
77,Southampton,Chelsea,0,3,8
78,Tottenham,Cardiff,1,0,8
79,Watford,Bournemouth,0,4,8


In [32]:
def single_team_db(predicted_round: int):
    games_df = games_allowed[games_allowed['round'] < predicted_round]
    host_df = games_df.groupby('host').agg('sum').reset_index()
    host_df.drop(labels=['round'], axis=1)
    guest_df = games_df.groupby('guest').agg('sum').reset_index()
    guest_df.drop(labels=['round'], axis=1)

    tot_each_team_df = DataFrame(data={
        "team": host_df.host,
        "GF_as_any": host_df.goals_host + guest_df.goals_guest,
        "GA_as_any": host_df.goals_guest + guest_df.goals_host,
        "GF_as_host": host_df.goals_host,
        "GA_as_host": host_df.goals_guest,
        "GF_as_guest": guest_df.goals_guest,
        "GA_as_guest": guest_df.goals_host,
    })
    tot_each_team_df.sort_values(by=['team'], ascending=True)
    return tot_each_team_df

final_single_team_db = single_team_db(predicted_round=1 + games_allowed['round'].max())
final_single_team_db

,team,GF_as_any,GA_as_any,GF_as_host,GA_as_host,GF_as_guest,GA_as_guest
0,Arsenal,54,33,34,9,20,24
1,Bournemouth,33,47,20,17,13,30
2,Brighton,24,37,11,12,13,25
3,Burnley,30,49,16,23,14,26
4,Cardiff,27,45,17,25,10,20
5,Chelsea,41,30,23,7,18,23
6,Crystal Palace,34,37,13,13,21,24
7,Everton,37,31,19,16,18,15
8,Fulham,21,59,14,27,7,32
9,Huddersfield,15,45,7,20,8,25


In [33]:
HOST_COLUMNS_NAMES = {
    'team': 'team1',
    'GF_as_any': 'GF1_as_any',
    'GA_as_any': 'GA1_as_any',
    'GF_as_host': 'GF1_as_host',
    'GA_as_host': 'GA1_as_host',
}

GUEST_COLUMNS_NAMES = {
    'team': 'team2',
    'GF_as_any': 'GF2_as_any',
    'GA_as_any': 'GA2_as_any',
    'GF_as_guest': 'GF2_as_guest',
    'GA_as_guest': 'GA2_as_guest',
}

In [34]:
def matches_to_predict(predicted_round: int) -> games_allowed:
    return games_allowed[games_allowed['round'] == predicted_round]

In [63]:
MIN_ROUND_PREDICTED = 8
MAX_ROUND_PREDICTED = games_allowed['round'].max()


def final_df(predicted_round: int):
    matches = matches_to_predict(predicted_round=predicted_round)

    single_team_sums = single_team_db(predicted_round=predicted_round)

    sums_up_to_round = None
    for ind, row in matches.iterrows():
        host = row['host']
        guest = row['guest']

        host_stats: DataFrame
        guest_stats: DataFrame

        host_stats = single_team_sums.loc[single_team_sums['team'] == host]
        guest_stats = single_team_sums.loc[single_team_sums['team'] == guest]
        host_stats = host_stats.rename(columns=HOST_COLUMNS_NAMES)
        guest_stats = guest_stats.rename(columns=GUEST_COLUMNS_NAMES)

        # there are no stats (sums) for initial rounds
        # since not all teams play in a single round
        if not host_stats.shape[0]:
            continue

        host_stats.index = [0]
        guest_stats.index = [0]

        line = pd.concat([host_stats, guest_stats], axis=1)
        if sums_up_to_round is not None:
            sums_up_to_round = pd.concat([sums_up_to_round, line], axis=0)
        else:
            sums_up_to_round = line

    return sums_up_to_round

PREDICTED_ROUND = 15

# initial value
all_sums_up_to_round = final_df(predicted_round=MIN_ROUND_PREDICTED)
for r in range(MIN_ROUND_PREDICTED + 1, PREDICTED_ROUND + 1):
    next_round_sums = final_df(r)
    all_sums_up_to_round = pd.concat([all_sums_up_to_round, next_round_sums], axis=0)

all_sums_up_to_round.reset_index(drop=True, inplace=True)
all_sums_up_to_round.to_excel('output.xlsx')

final_df(predicted_round=PREDICTED_ROUND).reset_index(drop=True).to_excel('round_output.xlsx')